# XTERM.JS le retour

---

## Introduction

> Dans cette troisième partie, nous allons d'abord recycler le code web assembly écrit pour faire fonctionner xterm.js ensuite, le but sera de pouvoir écrire plusieurs chaines de caractère au serveur depuis le terminal, ainsi le serveur nous répondra via le terminal aussi.

## La réinstallation de xterm

**Le code présent dans cette section devra être simplement du copier/coller, on ne va pas refaire et réfléchir deux fois aux mêmes choses.**

- Exécuter npm install xterm depuis la racine du projet

`npm install xterm`

*Le fichier package.json aura une nouvelle dépendance, xterm version 5.*

- Récupérer la feuille de style

D'abord, il faut ajouter deux dépendances au fichier package.json

```
    "css-loader": "^6.7.4",
    "style-loader": "^3.3.3",
```

Ensuite exécuter la commande npm install

`npm install`

Puis finalement ajouter l'import du fichier .css depuis index.js

**le fichier index.js devrait finalement ressembler à cela :**
```
import("./node_modules/xterm/css/xterm.css").catch(console.error);
import('./pkg').catch(console.error);
```

- Rajouter une section module dans le fichier `webpack.config.js` :

```
    module: {
        rules: [{
            test: /\.css$/,
            use: [
                'style-loader',
                'css-loader'
            ]
       }]
    },

```

- Modifications à apporter au fichier `lib.rs` :

```
#[wasm_bindgen(module = "xterm")]
extern "C" {
    type Terminal;

    #[wasm_bindgen(constructor)]
    fn new() -> Terminal;

    #[wasm_bindgen(method)]
    fn open(this: &Terminal, element: &web_sys::HtmlElement);

    #[wasm_bindgen(method)]
    fn write(this: &Terminal, chain: &str);

    #[wasm_bindgen(method)]
    fn attachCustomKeyEventHandler(this: &Terminal, closure: &Closure<dyn FnMut(web_sys::KeyboardEvent)>);
} 
```

- Ajouter la feature `KeyboardEvent` au fichier `cargo.toml` dans les dépendances de web-sys :

```
[dependencies.web-sys]
version = "0.3.63"
features = [
  'Document',
  'Element',
  'HtmlElement',
  'Node',
  'Window',
  'MessageEvent',
  'WebSocket',
  'ErrorEvent',
  'KeyboardEvent'
] 
```

- Création du DOM de la page depuis `lib.rs`. Rajouter ces lignes au début de la fonction principale :

```
    // Use `web_sys`'s global `window` function to get a handle on the global
    // window object.
    let window = web_sys::window().expect("no global `window` exists");
    let document = window.document().expect("should have a document on window");
    let body = document.body().expect("document should have a body");

    let terminal_div = document.create_element("div").unwrap();
    body.append_child(&terminal_div).unwrap();   
```

- Création et ouverture d'une instance xterm. Rajoutez ensuite ces 3 lignes :

```
    let terminal = Terminal::new();
    terminal.open(&terminal_div.dyn_into::<web_sys::HtmlElement>().unwrap());
    terminal.write("Hello from \x1B[1;3;31mxterm.js\x1B[0m $ "); 
```

**La page web devrait ressembler à cela.**

![PAGE](./pictures/page.png)

## Faire fonctionner le terminal avec la websocket

- **Vous devez redéfinir la callback attachée a `attachCustomKeyEventHandler` de façon à ce que les chaines soient envoyées au serveur lors de l'appui sur la touche `Entrer`.**

- **Les messages reçus par le client doivent s'afficher dans le terminal.**